In [102]:
import jinja2
import json
import re

In [8]:
#main.tf:

main = """
provider "snowflake" {
  account = "your-snowflake-account"
  region  = "your-snowflake-region"
}
"""

In [9]:
#users.tf
user_template = jinja2.Template(
"""
locals {
  users = {
    "{{user_name}}" = {
      login_name = "{{user_name}}"
      role       = "{{role}}"
      namespace  = "{{database}}.{{schema}}"
      warehouse  = "{{warehouse}}"
    }
    "CHEESE" = {
      login_name = "CHEESE_DATAENGINEER@MACANDCHEESE.COM"
      role       = "TRANSFORMER"
      namespace  = "DEV.PUBLIC"
      warehouse  = "TRANSFORMER_WH"
    }
    "STITCH" = {
      login_name = "STITCH@MACANDCHEESE.COM"
      role       = "LOADER"
      namespace  = "DEV.PUBLIC"
      warehouse  = "LOADER_WH"
    }
  }
}

resource "snowflake_user" "user" {
  for_each             = local.users
  name                 = each.key
  login_name           = each.value.login_name
  default_role         = each.value.role
  default_namespace    = each.value.namespace
  default_warehouse    = each.value.warehouse
  must_change_password = false
}
"""
)

In [10]:
#roles.tf
role_template = jinja2.Template("""
locals {
  roles = {
    "LOADER" = {
      comment = "Owns the tables in raw schema"
    }
    "TRANSFORMER" = {
      comment = "Has query permissions on tables in raw schema and owns tables in the analytics schema."
    }
  }
}

resource "snowflake_role" "role" {
  for_each = local.roles
  name     = each.key
  comment  = each.value.comment
}

locals {
  schemas = {
    "RAW" = {
      database = "DEV"
      comment = "contains raw data from our source systems"
      usage_roles = ["TRANSFORMER"]
      all_roles = ["LOADER"]
    }
    "ANALYTICS" = {
      database = "DEV"
      comment = "contains tables and views accessible to analysts and reporting"
      usage_roles = []
      all_roles = ["TRANSFORMER"]
    }
  }
}

resource "snowflake_schema" "schema" {
  for_each = local.schemas
  name     = each.key
  database = each.value.database
  comment  = each.value.comment
}

resource "snowflake_schema_grant" "schema_grant_usage" {
  for_each      = local.schemas
  schema_name   = each.key
  database_name = each.value.database
  privilege     = "USAGE"
  roles         = each.value.usage_roles
  shares        = []
}

resource "snowflake_schema_grant" "schema_grant_all" {
  for_each      = local.schemas
  schema_name   = each.key
  database_name = each.value.database
  privilege     = "ALL"
  roles         = each.value.all_roles
  shares        = []
}
""")

In [11]:
#schema.tf
schema_template = jinja2.Template("""
locals {
  schemas = {
    "RAW" = {
      database = "DEV"
      comment = "contains raw data from our source systems"
      usage_roles = ["TRANSFORMER"]
      all_roles = ["LOADER"]
    }
    "ANALYTICS" = {
      database = "DEV"
      comment = "contains tables and views accessible to analysts and reporting"
      usage_roles = []
      all_roles = ["TRANSFORMER"]
    }
  }
}

resource "snowflake_schema" "schema" {
  for_each = local.schemas
  name     = each.key
  database = each.value.database
  comment  = each.value.comment
}

resource "snowflake_schema_grant" "schema_grant_usage" {
  for_each      = local.schemas
  schema_name   = each.key
  database_name = each.value.database
  privilege     = "USAGE"
  roles         = each.value.usage_roles
  shares        = []
}

resource "snowflake_schema_grant" "schema_grant_all" {
  for_each      = local.schemas
  schema_name   = each.key
  database_name = each.value.database
  privilege     = "ALL"
  roles         = each.value.all_roles
  shares        = []
}
""")

In [140]:
class Toboggan:
    """
    inputs: users, roles, queries, syntax
    outputs: users.tf, roles.tf, queries.json, main.tf, variables.tf
    """
    def __init__(self, mode='create', main_tf=None, warehouses_file=None, users_file=None, roles_file=None, schemas_file=None, tables_file=None, queries_file=None, queries_syntax=None):
        self.mode = mode
        self.sep = ' ------------------------------ '
        self.main_tf = main_tf
        self.warehouses_file = warehouses_file
        self.users_file = users_file
        self.roles_file = roles_file
        self.schemas_file = schemas_file
        self.tables_file = tables_file
        self.queries_file = queries_file
        self.queries_syntax = queries_syntax
        self.warehouses = []
        self.roles = []
        self.databases = []
        self.schemas = []
        self.tables = []
        self.users = []
        self.queries = []
        
#         if queries_file:
#             self.users = self.read_file(users_file)
#         else:
#             self.users = self.intake_users()
    
    @staticmethod
    def clean_text(start_text, mode="lower"):
        """
        start_text: str : input text
        return: str : corrected text
        """
        if mode == 'lower':
            text = start_text.lower()
        else:
            text = start_text.upper()
        text = text.replace(" ", "_")
        text = text.replace("-", "_")
        text = re.sub(r'\W+', '', text)

        if start_text != text:
            print(f"Standardized naming to {text}")
        return text
    
    @staticmethod
    def read_file(filename):
        if '.json' in filename:
            with open('filename') as json_file:
                data = json.load(json_file)
                print(data)
        else:
            with open('filename') as f:
                data = f.read().splitlines()
        
        return data
    
    def intake_warehouse(self, warehouses):
        """
        warehouses: list: list of warehouses or empty list
        return: list: list of warehouses populated with at least 1 warehouse
        """
        print(f"\n{self.sep}WAREHOUSES{self.sep}\n")
        """
        resource "snowflake_warehouse" "warehouse_terraform" {
              name              =   "dev_wh"
              warehouse_size    =   "SMALL"
              auto_resume       =   false
              auto_suspend      =   600
              comment           =   "terraform development warehouse"
        }
        """
        warehouse_sizes = ['XSMALL', 'SMALL', 'MEDIUM', 'LARGE', 'XLARGE', 'XXLARGE', 'XXXLARGE', 'X4LARGE']
        
        while True:
            ans = input("Do you want to add a warehouse? y/n\n")
            if ans.lower() in ['y', 'yes']:
                warehouse = self.clean_text(input("What is the warehouse name?"))
                
                if warehouse in [x['name'] for x in warehouses]:
                    print("Warehouse already added, please use unique names.\n")
                    continue      
                
                while True:
                    warehouse_size = input(f"""What size should the warehouse be?\n 
                                                   1.'XSMALL'\n
                                                   2.'SMALL'\n
                                                   3.'MEDIUM'\n
                                                   4.'LARGE'\n
                                                   5.'XLARGE'\n
                                                   6.'XXLARGE'\n
                                                   7.'XXXLARGE'\n
                                                   8.'X4LARGE'\n
                                           """)

                    if warehouse_size not in ['1','2','3','4','5','6','7','8']:
                        print('Use a number between 1 and 8.')
                        continue
                    else:
                        warehouse_size = warehouse_sizes[int(warehouse_size) - 1]
                        break
                
                while True:
                    auto_resume = input(f"Should this warehouse start when queuried automatically?\n1. true\n2. false\n")
                    
                    if auto_resume not in ['1', '2']:
                        print("Use 1 or 2.\n")
                        continue
                    else:
                        if auto_resume == '1':
                            auto_resume = 'true'
                        else:
                            auto_resume = 'false'
                        break
                
                while True:
                    auto_suspend = input(f"How long for the warehouse to suspend after inactivity? (1 - 3600 seconds)\n")
                    try:
                        if int(auto_suspend) > 3601 or int(auto_suspend) < 1:
                            print("Use a number between 1 and 3600\n")
                            continue
                            
                    except:
                        print("Use a number between 1 and 3600\n")
                        continue
                
                comment = input(f"Leave a descriptive comment to describe {warehouse}:\n")
                
                warehouses.append({'name':warehouse, 'warehouse_size':warehouse_size, 'auto_resume':auto_resume, 'auto_suspend':auto_suspend, 'comment':comment})
                
            elif ans.lower() in ['n', 'no']:
                if len(roles) < 1:
                    print("You have to add at least one warehouse.")
                else:
                    break
            else:
                print('Please use yes, y, no, or n\n')
        
        return warehouses
        
        
    def intake_roles(self, roles):
        """
        roles: list: list of roles or empty list
        return: list : list of roles populated with at least 1 role
        """
        print(f"\n{self.sep}ROLES{self.sep}\n")
        roles.append({"PUBLIC":"Default Role"})

        while True:
            ans = input("Do you want to add a role? y/n\n")
            if ans.lower() in ['y', 'yes']:
                role = self.clean_text(input("What is the role name?"))
                #TODO name logic
                
                if role in [x['name'] for x in roles]:
                    print("Role already added, please use unique names.\n")
                    continue      
                
                comment = input(f"Leave a descriptive comment to describe {role}:\n")
                
                roles.append({'name':role,'comment':comment})
            elif ans.lower() in ['n', 'no']:
                if len(roles) < 1:
                    print("You have to add at least one role.")
                else:
                    break
            else:
                print('Please use yes, y, no, or n\n')
        
        return roles
    
    def intake_databases(self, databases):
        """
        databases: list :list of databases or empty list
        returns: list : list of databases populated with at least 1 database
        """
        print(f"\n{self.sep}DATABASES{self.sep}\n")
        databases = []

        while True:
            ans = input("Do you want to add a database? y/n\n")
            if ans.lower() in ['y', 'yes']:
                database = self.clean_text(input("What is the database name?"), mode='upper')
                
                if database in databases:
                    print("Database already added, please use unique names.\n")
                    continue      

                databases.append({'name': database})
                
            elif ans.lower() in ['n', 'no']:
                if len(databases) < 1:
                    print("You have to add at least one database.")
                else:    
                    break
            else:
                print('Please use yes, y, no, or n\n')
        
        return databases

    def intake_schemas(self, schemas):
        """
        schemas: list of schemas or empty list
        returns: list : list of schemas populated with at least 1 schema
        """
        print(f"\n{self.sep}SCHEMAS{self.sep}\n")

        while True:
            ans = input("Do you want to add a schema? y/n\n")
            if ans.lower() in ['yes', 'y']:
                name = self.clean_text(input("What is the schema name?"), mode='upper')
                if name in [x['name'] for x in schemas]:
                    print("Schema already added, please use unique names.\n")
                    continue
                
                comment = input(f"Leave a descriptive comment to describe {name}:\n")
                
                out_dict = {'name':name, 'comment':comment, 'database': None, 'usage_access':[], 'all_access':[]}
                while True:
                    database = input(f"Which database should {name} be created in? {self.databases}\n")

                    if database not in self.databases:
                        print(f"Please use a valid value from {self.databases}.\n")
                        continue

                    elif database in schema_databases:
                        print("Database already added, please use unique names.\n")
                        continue        

                    else:
                        out_dict['database'] = database
                        database_roles = []
                        while True:
                            ans = input("Do you want to add a role to access objects in {database}.{name}? y/n\n")
                            if ans.lower() in ['yes', 'y']:
                                role = input(f"What role should have access to objects in {database}.{name}? {self.role_names}\n")

                                if role not in self.role_names:
                                    print(f"Please use a valid value from {self.role_names}.\n")
                                    continue

                                elif role in database_roles:
                                    print("Role already added, please use unique names.\n")
                                    continue    
                                
                                elif ans.lower() in ['n', 'no']:
                                    if len(database_roles) < 1:
                                        print("The schema must have at least 1 role.")
                                    else:
                                        break

                                else:
                                    database_roles.append(role)
                                    while True:
                                        access_type = input(f"""What type of access would you like to grant to {role} on {database}.{name}?\n
                                                            1. USAGE\n
                                                            2. ALL\n
                                                            Enter 1 or 2:\n
                                                            """)
                                        if access_type not in ["1", "2"]:
                                            print("Enter 1 or 2.")
                                            continue
                                        else:
                                            if access_type == '1':
                                                access_level = 'USAGE'
                                                out_dict['usage_access'] = out_dict['usage_access'].append(role)
                                            else:
                                                access_level = 'ALL'
                                                out_dict['all_access'] = out_dict['all_access'].append(role)

                                            print(f"Added {role} role to {access_level} roles for {database}.{name}")

                                            break
                                    schemas.append(out_dict)
                                    continue

                            elif ans.lower() in ['n', 'no']:
                                if len(database_roles) < 1:
                                    print("The schema must have at least one role.")
                                else:
                                    break
                            else:
                                print('Please use yes, y, no, or n\n')

            elif ans.lower() in ['n', 'no']:
                if len(schemad) < 1:
                    print("You must define at least 1 schema.")
                    continue
                else:
                    break
            else:
                print('Please use yes, y, no, or n\n')
                continue
            
        return schemas

    def intake_tables(self):
        pass

    def intake_users(self, users):
        """
        users: list of users or empty list
        returns: list : list of users populated with at least 1 user
        """
        print(f"\n{self.sep}USERS{self.sep}\n")

        while True:
            ans = input("Do you want to add a user? y/n\n")
            if ans.lower() in ['yes', 'y']:
                user = {}
                name = self.clean_text(input("What is the user's username?"))
                if name in [x['name'] for x in users]:
                    print(f"User '{name}' already added, please use unique names.\n")
                    continue
                
                else:
                    user['name'] = name
                
                while True:
                    if role not in self.role_names:
                        print(f"Please use a valid value from {self.role_names}.\n")
                        continue                      

                else:
                    user['role'] = role
                    
                warehouses = [x['name'] for x in self.warehouses]
                while True:
                    print(f"What warehouse should {name} have?\n")
                    i = 1
                    for option in warehouses:
                        print(f"{i}. {option})
                        i+= 1
                    warehouse = input(f"Choose an option between 1 and {i-1}")

                    try:
                        warehouse = schemas[int(warehouse) - 1]
                        user['warehouse'] = warehouse
                        break
                    except:
                        print(f"Choose an option between 1 and {i-1}")
                        continue
                
                schemas = [f"{x['database']}.{x['schema']}" for x in self.schemas]
                i=1
                while True:
                    print(f"What default database and schema should {name} have?\n")
                    for option in schemas:
                        print(f"{i}. {option})
                        i+= 1
                    namespace = input(f"Choose an option between 1 and {i-1}")
                              
                    try:
                        namespace = schemas[int(namespace) - 1]
                        user['namespace'] = namespace
                        break
                    except:
                        print(f"Choose an option between 1 and {i-1}")
                        continue
                
            elif ans.lower() in ['n', 'no']:
                if len(users) < 1:
                    print("You have to add at least one user.")
                else:
                    break
            else:
                print('Please use yes, y, no, or n\n')
        
        return users
    
    def intake_queries(self):
        pass

    def init_doc(self):
        """
        provider "snowflake" {
          account = "your-snowflake-account"
          region  = "your-snowflake-region"
        }
        """
        pass
    
    def create_warehouses(self):
        """
        Add warehouses to main.tf
        self.warehouses : {'name':warehouse, 'warehouse_size':warehouse_size, 'auto_resume':auto_resume, 'auto_suspend':auto_suspend, 'comment':comment}
        """
        
        out_str = ""
        
        warehouse_template = jinja2.Template("""
        resource "snowflake_warehouse" "warehouse_terraform" {
              name              =   "{{name}}"
              warehouse_size    =   "{{warehouse_size}}"
              auto_resume       =   {{auto_resume}}
              auto_suspend      =   {{auto_suspend}}
              comment           =   "{{comment}}"
        }""")
        
        for warehouse in self.warehouses:
            out_str += warehouse_template.render(name=warehouse['name'],
                                                 warehouse_size=warehouse['warehouse_size'], 
                                                 auto_resume=warehouse['auto_resume'], 
                                                 auto_suspend=warehouse['auto_suspend'],
                                                 comment=warehouse['comment']) + "\n"
        
        return out_str
    
    def create_roles(self):
        """
        create roles.tf if not provided
        self.roles : [{'name': 'role_name', 'comment':'description'}]
        """
        role_template = jinja2.Template("""
            locals {
              roles = {
               {{role_blob}}
              }
            }
            
            resource "snowflake_role" "role" {
              for_each = local.roles
              name     = each.key
              comment  = each.value.comment
            }

            resource "snowflake_role_grants" "role_grant" {
              for_each  = local.roles
              role_name = each.key
              users     = each.value.users
              roles     = []
            }
            """)

        role_insert_template = jinja2.Template("""
                                                "{{role}}" = {
                                                  comment = "{{comment}}"
                                                  users = {{users}}
                                                }
                                                """)
        
        role_blob = """"""
        
        for role in self.roles:
            out_dict = item.copy()
            out_dict['users'] = []
            for user in self.users:
                if role['name'] in user['roles']:
                    out_dict['users'] = out_dict['users'].appent(user['name'])
            role_blob += role_insert_template.render(role=item['name'], comment=item['comment']).replace("'",'"')
                                
        out_str = role_template.render(roles_blob=roles_blob)
        
        with open('roles.tf', 'w+') as f:
            f.write(out_str)
        
    def create_schemas(self):
        """
        self.schemas : [{'name':name, 'comment':comment, 'database': database, 'usage_access':[roles], 'all_access':[roles]}]
        """
        schema_template = jinja2.Template(
            """
            locals {
              schemas = {
                {{schema_blob}}
              }
            }
            resource "snowflake_schema" "schema" {
              for_each = local.schemas
              name     = each.key
              database = each.value.database
              comment  = each.value.comment
            }

            resource "snowflake_schema_grant" "schema_grant_usage" {
              for_each      = local.schemas
              schema_name   = each.key
              database_name = each.value.database
              privilege     = "USAGE"
              roles         = each.value.usage_roles
              shares        = []
            }

            resource "snowflake_schema_grant" "schema_grant_all" {
              for_each      = local.schemas
              schema_name   = each.key
              database_name = each.value.database
              privilege     = "ALL"
              roles         = each.value.all_roles
              shares        = []
            }
            """)
        
        schema_insert_template = jinja2.Template(
            """
            "{{schema}}" = {
                  database = "{{database}}"
                  comment = "{{comment}}"
                  usage_roles = {{usage_roles}}
                  all_roles = {{all_roles}}
                }\n
            """
        )
            
        schema_blob = """"""
            
        for item in self.schemas:
            schemas_blob += schema_insert_template.render(schema=item['name'], database=item['database'], usage_roles=item['usage_roles'], all_roles=item['all_roles']).replace("'",'"')
                                
        out_str = schema_template.render(schemas_blob=schema_blob)
        
        with open('roles.tf', 'w+') as f:
            f.write(out_str)

    def create_users(self):
        """
        self.users : [{'name': 'role'}]
        """
        user_template = jinja2.Template(
                """
                locals {
                  users = {
                    {{user_blob}}
                  }
                }

                resource "snowflake_user" "user" {
                  for_each             = local.users
                  name                 = each.key
                  login_name           = each.value.login_name
                  default_role         = each.value.role
                  default_namespace    = each.value.namespace
                  default_warehouse    = each.value.warehouse
                  must_change_password = false
                }
                """)
        
        user_insert_template = jinja2.Template("""
                                                "{{user}}" = {
                                                          login_name = "{{user}}"
                                                          role       = "{{role}}"
                                                          namespace  = "{{namespace}}"
                                                          warehouse  = "{{warehouse}}"
                                                        }
                                              """)
        
        user_blob = """"""
        
        
#         for item in self.users:
#             for k, v in item.items():             
#                 user_blob += roles_indv_template.render(login_name=k, role=v, namespace=v, warehouse) + "\n"
                                
        out_str = role_template.render(roles_blob=roles_blob)
        for user in self.users:
            user_blob += user_insert_template.render(user=user, role="") #TODO
        
        with open('users.tf', 'w+') as f:
            f.write()
    
    def convert_sql(self, sql_syntax):
        """
        convert a query from another syntax to snow sql
        """
        pass
    
    def create_main_tf(self): 
        """
        create terraform
        """
        acct = input("What is your snowflake account address?\n")
        region = input("What is your snowflake account address?\n")
        main_template = jinja2.Template("""provider "snowflake" {
                                                      account = "{{acct}}"
                                                      region  = "{{region}}"
                                                    }""")
        out_str = main_template.render(acct=acct, region=region) + "\n"
        
        out_str += self.create_warehouses()
        
        with open("main.tf", "w") as f:
            f.write(out_str)

    def main(self):
        if self.main_tf:
            #TODO logic here
            pass
        else:
            self.create_main_tf()
        
        if self.warehouses_file:
            pass
        else:
            self.warehouses = self.intake_warehouses(self.warehouses)
        
        self.roles = self.intake_roles(self.roles)
        self.role_names = [x['name'] for x in self.roles]
        
        if self.roles_file:
            self.roles = self.read_file(roles_file)
        
        self.roles = self.intake_roles(self.roles)
        self.role_names = [x['name'] for x in self.roles]

#         if tables_file:
#             self.tables = self.read_file(tables_file)
#         else:
#             self.tables = self.intake_tables()

        if self.databases_file:
            self.databases = self.read_file(databases_file)
        
        self.databases = self.intake_databases(self.databases)
        self.database_names = [x['name'] for x in self.databases]

        if self.schemas_file:
            self.schemas = self.read_file(schemas_file)
            
        self.schemas = self.intake_schemas(self.schemas)
        self.schema_names = [x['name'] for x in self.schemas]

        if self.users_file:
            self.users = self.read_file(users_file)
        
        self.users = self.intake_users(self.users)
        self.user_names = [x['name'] for x in self.users]

#     def apply_regex_sub(regex, expression, sub_string):
#         import re
#         p = re.compile(regex, re.MULTILINE) 
#         return re.sub(p, sub_string, expression)

#     def mysql_to_snowflake(mysql_ddl):
#         result = apply_regex_sub(r'`', mysql_ddl, u\"\") # Remove `
#         result = apply_regex_sub(r'((--(.+)?)|(/\\*(.+)))\\n?', result, u\"\") # Remove all comments fields
#         result = apply_regex_sub(r'(DROP(.)+)\\n', result, \"\") # Remove DROP Table reference
#         result = apply_regex_sub(r'\\sDEFAULT(.+,)', result, u\",\") # Remove DEFAULT
#         result = apply_regex_sub(r'\\s((NOT\\sNULL)|NULL)', result, u\"\") # Remove NULL
#         result = apply_regex_sub(r'(((enum|varchar|nvarchar)\\(['0-9a-zA-Z,]+\\))|(text))(.)+\", result, u\"STRING,\") # STRING data types
#         result = apply_regex_sub(r'(tiny)?int\\([0-9a-zA-Z,]+\\)(\\s(unsigned))?', result, u\"NUMBER\") # NUMBER data types
#         result = apply_regex_sub(r'datetime', result, u\"TIMESTAMP_LTZ\") # TIMESTAMP_LTZ data types
#         result = apply_regex_sub(r'\\s\\s(((PRIMARY)|(UNIQUE))\\s)?KEY(.+)\\n', result, u\"\") # Strip KEYS
#         result = apply_regex_sub(r'AUTO_INCREMENT',result,\"\") #Strips AUTO_INCREMENT
#         result = apply_regex_sub(r'\\s\\s(CONSTRAINT\\s)(.+)\\n', result, u\"\") # Strip CONSTRAINTS
#         result = apply_regex_sub(r',?\\n\\)(.+)', result, u\"\\n);\") # Clean closing bracket
#         result = apply_regex_sub(r'^(?:[\\t ]*(?:\\r?\\n|\\r))+', result, \"\") # Discard blank lines
#         return result

SyntaxError: positional argument follows keyword argument (<ipython-input-140-58c6662fe098>, line 525)

In [97]:
z = Toboggan(mode='create')

ROLES



Do you want to add a role? y/n
 y
What is the role name? Spaces and certain punctuation removed. Developer Access -> developer_access
 dev
Leave a descriptive comment to describe dev:
 devs
Do you want to add a role? y/n
 y
What is the role name? Spaces and certain punctuation removed. Developer Access -> developer_access
 dev


Role already added, please use unique names.



Do you want to add a role? y/n
 y
What is the role name? Spaces and certain punctuation removed. Developer Access -> developer_access
 qa
Leave a descriptive comment to describe qa:
 qas
Do you want to add a role? y/n
 n


USERS



...

Please use a valid value from ['qa', 'PUBLIC', 'dev'].



Do you want to add a role to ryan? y/n
 n
Do you want to add a user? y/n
 y
What is the user's user name? Spaces and certain punctuation removed. Ryan Bacastow -> ryan_bacastow
 ryan


User already added, please use unique names.



Do you want to add a user? y/n
 y
What is the user's user name? Spaces and certain punctuation removed. Ryan Bacastow -> ryan_bacastow
 test
Do you want to add a role to test? y/n
 n
Do you want to add a user? y/n
 n


TypeError: unhashable type: 'dict'

In [112]:
def clean_text(start_text):
    text = start_text.lower()
    text = text.replace(" ", "_")
    text = text.replace("-", "_")
    text = re.sub(r'\W+', '', text)
    

    if start_text != text:
        print(f"Standardized naming to {text}")
    return text

In [113]:
clean_text("this_IS_a-TE$ST")

Standardized naming to this_is_a_test


'this_is_a_test'

In [124]:
t.roles[0].keys()

dict_keys(['dev'])

In [131]:
names = [{"name":"test"}, {"name":"ryan"}]

In [134]:
x = [x['name'] for x in names]
x

['test', 'ryan']

In [ ]:
def test_create_users(self):
    """
    create users.json if not provided
    """
    pass

def test_create_roles(self):
    """
    create roles.json if not provided
    """
    pass

def test_create_associations(self):
    """
    create master role + user json
    """
    pass

def test_convert_sql(self, sql_syntax):
    """
    convert a query from another syntax to snow sql
    """
    pass

def test_create_tf(self): 
    """
    create terraform
    """
    pass


In [41]:
l = [{'a':'b'},{'c':'d'}]

In [53]:
all_roles = [k for k in l]

In [54]:
all_roles

[{'a': 'b'}, {'c': 'd'}]

In [57]:
list(all_keys)

['a', 'c']